# Ejercicio Hundir La Flota | Challenge yourself | Week 2
## Precurso DSNov20 - The Bridge

Se mantiene en español para que no se os olvide leer en español. 
Disculpen las molestias que esto pudiera ocasionarle como estudiante. 

En este ejercicio deberás programar el juego **hundir la flota** o [**batalla naval**](https://es.wikipedia.org/wiki/Batalla_naval_(juego)) 

El tablero de este juego será una matriz de 10x10 dimensiones, matriz sobre la que plamaremos el estado de cada turno. Cada matriz de información continente del estado en cada momento, se mostrará por pantalla.

![tablero](https://i.pinimg.com/474x/1c/66/ca/1c66ca8d767428dcd1c39285643f3987--yahoo-search.jpg)

**Las reglas son las siguientes:**

- Cada jugador deberá colocar en su tablero:
    - 4 barcos tamaño 2x1. 

    - 3 barcos tamaño 3x1.

    - 2 barcos tamaño 4x1.
    
    - 1 barco tamaño 5x1. 
    

- En cada turno, el jugador correspondiente debe elegir una casilla de la matriz (10x10) y ambos jugadores marcarán la casilla en la matriz.
- Si la casilla que se ha elegido es agua, será marcada como un una "**o**" y si la casilla que ha marcado es uno de los barcos, la casilla será marcada con una "**x**". 
- Los barcos han de ser mostrados con un "__*__" en la matriz. 
- Los espacios con agua serán mostrados como "**~**".

Antes de que comience la partida, cada jugador deberá crear su estado inicial. El estado inicial es una configuración de la matriz con todos los barcos colocados. Para ello, el programa debería preguntar, uno a uno, dónde se quiere colocar cada barco. El estilo de pregunta, ha de ser: 

- Inserte la posición del barco 1 (2x1). El usuario ha de escribir si se quiere insertar en la 4a fila, en horizontal y ocupando las últimas dos columnas: `4h9:10`.
- Inserte la posición del barco 2 (2x1). El usuario ha de escribir si se quiere insertar en la 8a fila, en horizontal y ocupando las dos primeras columnas: `8h1:2`.
- Inserte la posición del barco 3 (5x1). Si se quiere colocar en vertical empezando por la fila 1 a la 5 y columna 8: `8v1:5`.

También tendrás la opción de cargar una partida anteriormente definida en formato [JSON](https://www.w3schools.com/python/python_json.asp).

Para saber quien empieza a jugar, los jugadores deberán ponerse de acuerdo para que uno de ellos sea el que llame a una función que devuelva dos valores random del 1 al 6, de tipo entero. El primer valor que se muestre por pantalla será el correspondiente al jugador que llama a la función, el segundo será el del contrincante. Quien saque un valor más alto, empieza. 

Además, el programa guardará el nombre de los jugadores y el histórico, es decir, todos los estados de la partida. Para ello se utilizará el formato JSON.

Cuando un jugador haya creado su estado inicial, este estado también deberá ser guardado en formato JSON. Antes de que se inicie la partida, el otro jugador deberá cargar tu estado inicial y tú el suyo. El estado inicial del enemigo es el que usará tu programa para comprobar si las coordenadas que has utilizado han acertado en el blanco o no. 

El formato de preguntas por turno es: 

- `Inserte coordenadas a atacar:`. La respuesta válida será `0x8` --> fila 0, columna 8. 

Una vez insertada las coordenadas, deberá resultar en que aparezcan tu matriz y la matriz de estado de tu enemigo, esta última con la nueva coordenada marcada sin la flota excepto si el torpedo hubiera dado en el blanco. 

**A modo de resumen:**

1. Se eligen los nombres de jugador. ¡Recuerda no dejar espacios!

2. Se pregunta sobre las posiciones de tu flota. Una vez que se haya definido, el sistema debe guardar en JSON el estado de tu flota. OPCIONAL: Puedes cargar una partida a partir de un fichero JSON.

3. Se carga el estado inicial de tu enemigo a partir de un fichero JSON que te habrá definido previamente. El fichero estará en una carpeta llamada "Partidas_Batalla_Naval". Para cargar el fichero se debe introducir la ruta o nombre del fichero. 

4. Cuando ambos estéis en el mismo punto y se sepa quien es el primero, se debe escribir *primero* o *segundo*. Si eres primero, el sistema empezará preguntándote las coordenadas. Si eres segundo, el sistema preguntará por las coordenadas del rival. 

5. Se debe mostrar por pantalla dos matrices: una matriz 10x10 que representa los puntos que has ido marcando, que no mostrará la flota del enemigo excepto si algún barco ha sido dañado, y otra a la derecha que muestre tu flota con los puntos que hayan sido atacados por parte del contrincante. 

6. Cuando toda la flota de uno de los dos jugadores haya sido exterminada, se mostrará el ganador de la partida. Que si has sido tú el que ha programado este juego, ya habrás ganado incluso antes de empezar a jugar. 

In [21]:
# pprint to see the things prettier
import pprint as pp

 Ten en cuenta que lo ideal será que programes este juego en `.py` de modo que puedas ejecutar dicho programa por la terminal. 

In [22]:
# NUmber of players in teh game
NUM_PLAYERS = 2

# CELL Vars
WATER_CELL = "~"
BOAT_CELL = "*"
# REQUEST Vars
WATER_REQ = "o"
BOAT_REQ = "x"


# Create the initial table with all water
INIT_TABLE = [[WATER_CELL] * 10 for i in range(10)]
#pp.pprint(INIT_TABLE)

# Dict with the boat and characteristics
# The key is the name of the size, 
# the first value in the list is the number of cells,
# the second value in the list is the number of boats of this type
BOATS_MASTER_DICT = {
#   NAME: [NUMBER OF CELLS, NUMBER OF BOATS]
    "S": [2, 4],
    "M": [3, 3],
    "L": [4, 2],
    "XL": [5, 1], 
}

In [23]:
class Player():
    # Init value to know the order in the first round
    init_value = 0

    def __init__(self, name):
        self.name = name

class Boat():
    
    # Dict to cluster all the boats
    sizes_dict = {
        2: "S",
        3: "M",
        4: "L",
        5: "XL",
    }
    
    # Function to understand the orientation of the boat
    def get_orientation(self, pos):
        if "v" in pos.lower():
            return "VERTICAL"
        elif "h" in pos.lower():
            return "HORIZONTAL"
        else:
            return "ERROR"

    def __init__(self, player, size_num, pos):
        self.player = player
        self.size_num = size_num
        self.pos = pos
        self.size = self.sizes_dict[size_num]
        self.orientacion = self.get_orientation(pos) 

In [24]:
# Creare a ¡list to save the players
player_lst = []

# Iterate to create the players
for i in range(NUM_PLAYERS):
    name = ""
    # While the inserted value is not a correct name
    # repeat the input function
    while name.isdigit() or name is None or name == "":
        name = input("""
             Inserte el nombre del {0}º jugador y pulse enter.\n
             Recuerda no dejar espacios o se eliminaran""" \
            .format(i + 1))
        # Split the name and join the splitted to delete spaces
        name_splitted = name.split(" ")
        valid_name = "".join([ns.title() for ns in name_splitted])
        # Create the Players
        player = Player(valid_name)
        # Add the players to one list
        player_lst.append(player)
        # Print a welcome string for all the players
        welcome_str = "Bienvenido al juego {0}".format(player.name)
        print(welcome_str)

#[print(n.name) for n in player_lst]

Bienvenido al juego PatataMale
Bienvenido al juego GustavoMartinVela


In [26]:
# Iter over the players to save their boats
for p in player_lst:
    select_str = "Te toca elegir donde quieres poner tus barcos {0}".format(p.name)
    print(select_str)    

Te toca elegir donde quieres poner tus barcos PatataMale
Te toca elegir donde quieres poner tus barcos GustavoMartinVela


In [19]:
x = ["xzcac", "dsvsvs", "sdGYUI"]
"".join([t.title() for t in x])

'XzcacDsvsvsSdgyui'